In [7]:
# %load 101_user_based_statistics.py
#encoding:utf-8
import gc
import numpy as np
import pandas as pd
from tqdm import tqdm
from utils import BayesianSmoothing, load_pickle, dump_pickle, raw_data_path

train = pd.read_pickle('../data_valid/train.pkl')
test = pd.read_pickle('../data_valid/test.pkl')
df = pd.concat([train, test], axis=0)

df_feat = pd.DataFrame({'instance_id': df.instance_id, 'day': df.day})


user_cols = ['user_id', 'user_gender_id', 'user_age_level', 'user_occupation_id', 'user_star_level']
id_cols = ['instance_id', 'item_id', 'item_brand_id', 'item_city_id', 'shop_id']
df_cols = df.columns
# grade_cols = ['item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'shop_review_num_level',
#               'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description']
grade_cols = ['item_price_level']

def get_stats_target(df, group_column, target_column, drop_raw_col=False):
    df_old = df.copy()
    grouped = df_old.groupby(group_column)
    the_stats = grouped[target_column].agg(['mean', 'median', 'max', 'min', 'std', 'skew']).reset_index()
    the_stats.columns = [ group_column,
                            '_%s_groupby_%s_mean' % (target_column, group_column),
                            '_%s_groupby_%s_median' % (target_column, group_column),
                            '_%s_groupby_%s__max' % (target_column, group_column),
                            '_%s_groupby_%s__min' % (target_column, group_column),
                            '_%s_groupby_%s__std' % (target_column, group_column),
                            '_%s_groupby_%s__skew' % (target_column, group_column)
                        ]
    if drop_raw_col:
        df_old.drop(group_column, axis=1, inplace=True)

    return the_stats

def generate_basic_feats(df, feat_df):

    # ============================= 计算关于用户特征的id类特征的数量 =====================
    '''
    如：对于每个男性用户, 他所浏览的各种item_id的数量
    '''
    for user_col in (user_cols):
        cnt_result = df.groupby(user_col)[id_cols].nunique()
        cnt_result = cnt_result.add_prefix(user_col + '_').add_suffix('_cnt')
        cnt_result = cnt_result.reset_index()
        np.sum(feat_df.instance_id - pd.merge(df, cnt_result, on=user_col, how='left').instance_id)
        feat_df[user_col + '_count'] = pd.merge(df, cnt_result, on=user_col, how='left').iloc[:, -1].values
    print('the shape of feat_df {}'.format(feat_df.shape))
    # ============================ 计算关于用户特征的得分类特征的统计特征 ========================
    '''
    如：男性用户中item_price_level的平均数，中位数，最大值，最小值
    '''
    for user_col in (user_cols):
        # statistic feature
        for grade_col in grade_cols:
            the_stats = get_stats_target(df, user_col, grade_col, drop_raw_col=False)
            #stats_list = pd.concat([stats_list, the_stats], axis=1)
            tmp_df = pd.merge(df, the_stats, on=user_col, how='left')
            feat_df = pd.concat([feat_df, tmp_df.drop(df_cols, axis=1, inplace=True)] , axis=1)
    print('the shape of feat_df {}'.format(feat_df.shape))
    # ========================== 计算每个用户下的category2的个数 =========================
    '''
    如：对于每个男性用户，他所浏览的各种category2的数量
    '''
    for user_col in (user_cols):
        # category and property columns
        category_cnt = df.groupby(user_col)['item_category_list2'].nunique().reset_index()
        category_cnt.columns = [user_col, 'category2_groupby_{}_cnt'.format(user_col)]
        tmp_df = pd.merge(df, category_cnt, how='left', on=user_col)
        feat_df = pd.concat([feat_df, tmp_df.drop(df_cols, axis=1, inplace=True)], axis=1)

        del category_cnt
        gc.collect()
    print('the shape of feat_df {}'.format(feat_df.shape))
    return feat_df


df = generate_basic_feats(df, df_feat)
train_feat = df[(df['day'] >= 18) & (df['day'] <= 23)].drop('day', axis=1)
test_feat = df[df['day'] == 24].drop('day', axis=1)
print(train_feat.columns)
print('the shape of train {}'.format(train_feat.shape))
print('the shape of test {}'.format(test_feat.shape))
dump_pickle(train_feat, '../data_valid/train_feature/101_user_based_statistics.pkl')
dump_pickle(test_feat, '../data_valid/test_feature/101_user_based_statistics.pkl')

the shape of feat_df (478087, 7)
the shape of feat_df (478087, 7)
the shape of feat_df (478087, 7)
Index(['instance_id', 'user_id_count', 'user_gender_id_count',
       'user_age_level_count', 'user_occupation_id_count',
       'user_star_level_count'],
      dtype='object')
the shape of train (420676, 6)
the shape of test (57411, 6)


In [8]:
test_feat

,instance_id,user_id_count,user_gender_id_count,user_age_level_count,user_occupation_id_count,user_star_level_count
420676,2137051386648397465,1,3101,3325,3727,2797
420677,8636603765413923862,22,3770,3325,3276,2900
420678,345383580586199930,2,3770,3325,3276,2653
420679,8731890116021855314,3,3770,3184,3727,2769
420680,4313209774350468112,1,3101,3325,3276,2653
420681,3105730058528341650,6,3770,3325,3727,2797
420682,61137454093449956,1,3101,3325,3276,2900
420683,4900634264217133300,1,3770,2659,3727,2654
420684,3878416316017114607,12,3101,3325,3276,2797
420685,2091851204363732790,4,3770,3184,3727,2797
